In [2]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 20)

import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import linear_model

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

## Team dataset

In [3]:
def read_dataset(x):
    return pd.read_csv(os.path.join('..', 'datasets', x))

df_team_stats = read_dataset('top100_nba_historical_teams.csv')

In [4]:
df_team_stats

,Rk,Season,Tm,Lg,G,W,L,W/L%,MP,FG,...,ORtg,DRtg,eFG%,TOV%,ORB%,FT/FGA,oop_eFG%,oop_TOV%,oop_ORB%,oop_FT/FGA
0,1,1970-71,MIL*,NBA,82,66,16,0.805,19780.0,3972,...,NaN,NaN,0.509,NaN,NaN,0.226,0.424,NaN,NaN,0.210
1,2,1995-96,CHI*,NBA,82,72,10,0.878,19730.0,3293,...,115.2,101.8,0.517,13.1,36.9,0.217,0.482,16.1,28.9,0.222
2,3,1971-72,LAL*,NBA,82,69,13,0.841,19755.0,3920,...,NaN,NaN,0.490,NaN,NaN,0.260,0.432,NaN,NaN,0.177
3,4,1971-72,MIL*,NBA,82,63,19,0.768,19780.0,3813,...,NaN,NaN,0.498,NaN,NaN,0.232,0.420,NaN,NaN,0.217
4,5,1996-97,CHI*,NBA,82,69,13,0.841,19730.0,3277,...,114.4,102.4,0.511,12.5,35.9,0.199,0.471,14.8,30.7,0.196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,1984-85,BOS*,NBA,82,63,19,0.768,19705.0,3721,...,112.8,106.3,0.515,13.8,32.8,0.254,0.481,12.6,29.6,0.198
96,97,1994-95,ORL*,NBA,82,57,25,0.695,19930.0,3460,...,115.1,107.8,0.539,14.0,34.0,0.239,0.490,13.3,31.6,0.220
97,98,2011-12,OKC*,NBA,66,47,19,0.712,15990.0,2462,...,109.8,103.2,0.516,15.3,27.8,0.269,0.465,13.0,27.9,0.207
98,99,2012-13,LAC*,NBA,82,56,26,0.683,19730.0,3160,...,110.6,103.6,0.526,13.9,28.8,0.203,0.492,15.4,26.5,0.229


In [5]:
df.describe()

NameError: name 'df' is not defined

## Individual team dataset

In [28]:
df_indiv_team = read_dataset('95-96_chi.csv')

In [29]:
df_indiv_team

,Rk,G,Date,Unnamed: 3,Opp,W/L,Tm,Opp.1,FG,FGA,...,FT.1,FTA.1,FT%.1,ORB.1,TRB.1,AST.1,STL.1,BLK.1,TOV.1,PF.1
0,1,1,11/3/95,NaN,CHH,W,105,91,43,86,...,16,25,0.640,14,43,14,8,4,19,22
1,2,2,11/4/95,NaN,BOS,W,107,85,43,88,...,25,43,0.581,11,44,17,7,4,25,24
2,3,3,11/7/95,NaN,TOR,W,117,108,44,80,...,30,37,0.811,7,32,25,6,3,19,26
3,4,4,11/9/95,@,CLE,W,106,88,41,79,...,19,23,0.826,9,34,27,9,1,9,18
4,5,5,11/11/95,NaN,POR,W,110,106,40,81,...,18,30,0.600,15,40,23,4,1,17,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,78,4/14/96,@,CLE,W,98,72,34,70,...,6,10,0.600,11,33,20,7,3,16,15
78,79,79,4/16/96,@,MIL,W,86,80,33,85,...,9,15,0.600,10,42,14,7,1,9,22
79,80,80,4/18/96,NaN,DET,W,110,79,43,87,...,10,20,0.500,11,36,12,12,2,17,21
80,81,81,4/20/96,NaN,IND,L,99,100,40,90,...,26,41,0.634,11,39,15,14,3,17,16


## H1B data

In [8]:
df_h1b = read_dataset('H-1B_Disclosure_Data_FY16.csv')

In [9]:
df_h1b

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,...,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE,Unnamed: 40
0,I-200-09131-859003,CERTIFIED,3/10/16,3/16/16,H-1B,9/2/16,9/1/19,POINT INSIDE INC.,"500 108TH AVENUE NE, SUITE 2200",BELLEVUE,...,"$110,000.00",Year,N,N,BELLEVUE,KING,WA,98004,NaN,NaN
1,I-200-09180-429413,CERTIFIED,3/4/16,3/10/16,H-1B,8/22/16,8/21/19,"1NTEGER, LLC",1112 MONTANA AVENUE,SANTA MONICA,...,$0.00,Year,N,N,SANTA MONICA,LOS ANGELES COUNTY,CA,90401,NaN,NaN
2,I-200-09183-295355,CERTIFIED,3/11/16,3/17/16,H-1B,9/10/16,9/10/19,"OMRON AUTOMOTIVE ELECTRONICS, INC.",3709 OHIO AVENUE,ST. CHARLES,...,"$79,000.00",Year,N,N,NOVI,OAKLAND,MI,48377,NaN,NaN
3,I-200-09187-608762,CERTIFIED-WITHDRAWN,7/28/09,1/20/16,H-1B,10/1/09,10/1/12,"OPTIMAL SOLUTIONS & TECHNOLOGIES (OST), INC","2001 M STREET NW, SUITE # 3000",WASHINGTON,...,$0.00,Year,N,N,UNION TOWN,FAYETTE,PA,15401,7/29/09,NaN
4,I-200-09196-047365,CERTIFIED,3/17/16,3/23/16,H-1B,9/10/16,9/10/19,"UNION SPECIAL, LLC",ONE UNION SPECIAL PLAZA,HUNTLEY,...,"$65,000.00",Year,N,N,HUNTLEY,MCHENRY,IL,60142,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363226,I-200-16076-907489,CERTIFIED,3/16/16,3/22/16,H-1B,9/15/16,9/15/19,TECHSTAR SOLUTIONS INC,"12020 GREYWING SQ, T3",RESTON,...,$0.00,Year,N,N,RESTON,FAIRFAX,VA,20191,NaN,NaN
363227,I-200-16076-907503,CERTIFIED,3/16/16,3/22/16,H-1B,8/20/16,8/20/19,COLLABERA INC.,25 AIRPORT ROAD,MORRISTOWN,...,$0.00,Hour,N,N,DURHAM,DURHAM,NC,27703,NaN,NaN
363228,I-200-16076-907587,CERTIFIED,3/17/16,3/23/16,H-1B,9/16/16,9/16/19,NEOPEUTICS INC,400 OYSTER POINT BLVD,SOUTH SAN FRANCISCO,...,$0.00,Year,N,N,SOUTH SAN FRANCISCO,SAN MATEO,CA,94080,NaN,NaN
363229,I-200-16076-907848,CERTIFIED,3/16/16,3/22/16,H-1B,9/3/16,9/1/19,OPTIMUM SOLUTIONS INC.,1900 CAMPUS COMMONS DRIVE,RESTON,...,$0.00,Year,N,N,RESTON,FAIRFAX,VA,20191,NaN,NaN


In [11]:
df_h1b.CASE_STATUS.value_counts()

CERTIFIED              316181
CERTIFIED-WITHDRAWN     29818
WITHDRAWN               13080
DENIED                   4152
Name: CASE_STATUS, dtype: int64

In [12]:
df_h1b.describe()

/Users/gyzhang328/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,NAIC_CODE,TOTAL_WORKERS,FULL_TIME_POSITION,PW_WAGE_SOURCE_YEAR
count,363226.000000,363231.000000,0.0,363201.000000
mean,448223.317733,2.225399,NaN,2015.038593
std,193910.999838,6.416887,NaN,9.724249
min,211.000000,0.000000,NaN,0.000000
25%,NaN,1.000000,NaN,NaN
50%,NaN,1.000000,NaN,NaN
75%,NaN,1.000000,NaN,NaN
max,928120.000000,150.000000,NaN,7201.000000


In [22]:
df_h1b.loc[df_h1b.CASE_STATUS == 'DENIED']

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,...,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE,Unnamed: 40
77,I-200-11202-883062,DENIED,3/25/16,3/29/16,H-1B,9/24/16,9/23/19,"RMA ASSOCIATES, LLC",1050 N. TAYLOR ST.,ARLINGTON,...,$0.00,Year,N,N,ARLINGTON,ARLINGTON COUNTY,VA,22201,NaN,NaN
3673,I-200-13219-874996,DENIED,2/19/16,2/23/16,H-1B,8/5/16,8/5/19,DMI BIG DATA INSIGHTS LLC,4660 DUKE DR.,MASON,...,$0.00,Year,Y,N,MASON,WARREN,OH,45040,NaN,NaN
6480,I-200-14064-202867,DENIED,3/22/16,3/24/16,H-1B,9/18/16,9/18/19,INFINITE SPECTRUM LLC,4550 CHERRY CREEK SOUTH DRIVE,DENVER,...,$0.00,Hour,N,N,DENVER,ARAPAHOE,CO,80246,NaN,NaN
7630,I-200-14083-440661,DENIED,3/22/16,3/25/16,H-1B,9/20/16,8/30/19,SAISARATH TECHNOLOGIES LLC,2 TOWER CENTER BLVD,EAST BRUNSWICK,...,$0.00,Year,N,N,EAST BRUNSWICK,MIDDLESEX COUNTY,NJ,08816,NaN,NaN
8171,I-200-14113-278676,DENIED,12/16/15,12/21/15,H-1B,12/21/15,9/11/17,ARDENT TECHNOLOGIES INC,6234 FAR HILLS AVENUE,DAYTON,...,$0.00,Year,Y,N,DAYTON,MONTGOMERY,OH,45459,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362896,I-200-16076-876340,DENIED,3/16/16,3/17/16,H-1B,9/1/16,8/31/19,TECH-NET INC.,101 PARKSHORE DRIVE,FOLSOM,...,$0.00,Year,N,N,FOLSOM,SACRAMENTO,CA,95630,NaN,NaN
362938,I-200-16076-879904,DENIED,3/16/16,3/22/16,H-1B,7/1/16,7/1/19,INDIANA UNIVERSITY HEALTH BALL MEMORIAL PHYSIC...,2401 WEST UNIVERSITY AVE.,MUNCIE,...,"$240,000.00",Year,N,N,MUNCIE,DELAWARE,IN,47303,NaN,NaN
362962,I-200-16076-882376,DENIED,3/16/16,3/18/16,H-1B,9/15/16,9/15/19,"QSI SERVICES, INC.",5324 TWIN HICKORY ROAD,GLEN ALLEN,...,"$80,000.00",Year,Y,N,GLEN ALLEN,HENRICO,VA,23059,NaN,NaN
363009,I-200-16076-886090,DENIED,3/16/16,3/18/16,H-1B,9/15/16,9/15/19,KINGSBRIDGE PHARMACY CORP,145 W. KINGSBRIDGE RD,BRONX,...,$0.00,Hour,N,N,BRONX,NEW YORK,NY,10468,NaN,NaN


In [27]:
df_h1b.columns

Index([u'CASE_NUMBER', u'CASE_STATUS', u'CASE_SUBMITTED', u'DECISION_DATE',
       u'VISA_CLASS', u'EMPLOYMENT_START_DATE', u'EMPLOYMENT_END_DATE',
       u'EMPLOYER_NAME', u'EMPLOYER_ADDRESS', u'EMPLOYER_CITY',
       u'EMPLOYER_STATE', u'EMPLOYER_POSTAL_CODE', u'EMPLOYER_COUNTRY',
       u'EMPLOYER_PROVINCE', u'EMPLOYER_PHONE', u'EMPLOYER_PHONE_EXT',
       u'AGENT_ATTORNEY_NAME', u'AGENT_ATTORNEY_CITY', u'AGENT_ATTORNEY_STATE',
       u'JOB_TITLE', u'SOC_CODE', u'SOC_NAME', u'NAIC_CODE', u'TOTAL_WORKERS',
       u'FULL_TIME_POSITION', u'PREVAILING_WAGE', u'PW_UNIT_OF_PAY',
       u'PW_WAGE_SOURCE', u'PW_WAGE_SOURCE_YEAR', u'PW_WAGE_SOURCE_OTHER',
       u'WAGE_RATE_OF_PAY_FROM', u'WAGE_RATE_OF_PAY_TO', u'WAGE_UNIT_OF_PAY',
       u'H1B_DEPENDENT', u'WILLFUL_VIOLATOR', u'WORKSITE_CITY',
       u'WORKSITE_COUNTY', u'WORKSITE_STATE', u'WORKSITE_POSTAL_CODE',
       u'ORIGINAL_CERT_DATE', u'Unnamed: 40'],
      dtype='object')

## Fire data

In [35]:
df_fire = read_dataset('Fire_Incidents.csv')

In [36]:
df_fire

,Incident Number,Exposure Number,Address,Incident Date,Call Number,Alarm DtTm,Arrival DtTm,Close DtTm,City,Zipcode,...,Detector Effectiveness,Detector Failure Reason,Automatic Extinguishing System Present,Automatic Extinguishing Sytem Type,Automatic Extinguishing Sytem Perfomance,Automatic Extinguishing Sytem Failure Reason,Number of Sprinkler Heads Operating,Supervisor District,Neighborhood District,Location
0,9030109,0,310 Colon Av.,04/12/2009,91020273,04/12/2009 06:09:13 PM,04/12/2009 06:13:45 PM,04/12/2009 07:23:13 PM,SF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13067402,0,20 Lansdale Av,07/18/2013,131990117,07/18/2013 10:32:03 AM,07/18/2013 10:37:15 AM,07/18/2013 10:39:55 AM,SF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12044490,0,7th St. / Folsom St.,05/13/2012,121340051,05/13/2012 03:55:37 AM,05/13/2012 04:01:57 AM,05/13/2012 04:05:44 AM,SF,94103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,South of Market,"(37.7767460000297, -122.407844)"
3,13033326,0,2799 Pacific Av,04/09/2013,130990286,04/09/2013 04:34:07 PM,04/09/2013 04:39:31 PM,04/09/2013 05:20:27 PM,SF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11101416,0,Polk St. / Pine St.,11/01/2011,113050357,11/01/2011 06:07:45 PM,11/01/2011 06:10:17 PM,11/01/2011 06:11:09 PM,SF,94109.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,Nob Hill,"(37.7896190000297, -122.420497)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407709,16092668,0,Rose St/webster Street,08/22/2016,162353400,08/22/2016 08:34:29 PM,08/22/2016 08:36:19 PM,08/22/2016 09:38:54 PM,San Francisco,94102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,Hayes Valley,"(37.7727800000296, -122.428897)"
407710,16092672,0,Steiner St/union Street,08/22/2016,162353419,08/22/2016 08:40:00 PM,08/22/2016 08:44:24 PM,08/22/2016 08:44:28 PM,San Francisco,94123.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,Marina,"(37.7968790000297, -122.437138)"
407711,16092679,0,2547 California Street,08/22/2016,162353476,08/22/2016 08:55:45 PM,08/22/2016 09:00:17 PM,08/22/2016 09:05:45 PM,San Francisco,94115.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,Pacific Heights,"(37.7885290000297, -122.436251)"
407712,16092680,0,3901 Noriega Street,08/22/2016,162353493,08/22/2016 09:02:07 PM,08/22/2016 09:08:38 PM,08/22/2016 09:21:14 PM,San Francisco,94122.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Sunset/Parkside,"(37.7528900000297, -122.505487)"


In [37]:
df_fire.columns

Index([u'Incident Number', u'Exposure Number', u'Address', u'Incident Date',
       u'Call Number', u'Alarm DtTm', u'Arrival DtTm', u'Close DtTm', u'City',
       u'Zipcode', u'Battalion', u'Station Area', u'Box', u'Suppression Units',
       u'Suppression Personnel', u'EMS Units', u'EMS Personnel',
       u'Other Units', u'Other Personnel', u'First Unit On Scene',
       u'Estimated Property Loss', u'Estimated Contents Loss',
       u'Fire Fatalities', u'Fire Injuries', u'Civilian Fatalities',
       u'Civilian Injuries', u'Number of Alarms', u'Primary Situation',
       u'Mutual Aid', u'Action Taken Primary', u'Action Taken Secondary',
       u'Action Taken Other', u'Detector Alerted Occupants', u'Property Use',
       u'Area of Fire Origin', u'Ignition Cause', u'Ignition Factor Primary',
       u'Ignition Factor Secondary', u'Heat Source', u'Item First Ignited',
       u'Human Factors Associated with Ignition', u'Structure Type',
       u'Structure Status', u'Floor of Fire Origin', 

In [66]:
df_fire.shape[0] - df_fire['Civilian Injuries'].value_counts().sum()

0

In [107]:
def nan_count(df):
    i = 0
    while i < df.shape[1]:
        print df[[i]].columns[0]
        nan_count =407714 - df[df[[i]].columns[0]].value_counts().sum()
        print nan_count
        i=i+1

In [109]:
nan_count(df_fire)

Incident Number
0
Exposure Number
0
Address
106
Incident Date
0
Call Number
0
Alarm DtTm
0
Arrival DtTm
19
Close DtTm
0
City
1241
Zipcode
72401
Battalion
0
Station Area
530
Box
297822
Suppression Units
0
Suppression Personnel
0
EMS Units
0
EMS Personnel
0
Other Units
0
Other Personnel
0
First Unit On Scene
77905
Estimated Property Loss
281909
Estimated Contents Loss
274823
Fire Fatalities
0
Fire Injuries
0
Civilian Fatalities
0
Civilian Injuries
0
Number of Alarms
407714
Primary Situation
0
Mutual Aid
0
Action Taken Primary
0
Action Taken Secondary
65577
Action Taken Other
70983
Detector Alerted Occupants
68988
Property Use
337
Area of Fire Origin
380890
Ignition Cause
380891
Ignition Factor Primary
380892
Ignition Factor Secondary
383569
Heat Source
380890
Item First Ignited
380890
Human Factors Associated with Ignition
380893
Structure Type
397297
Structure Status
397366
Floor of Fire Origin
400379
Fire Spread
398128
No Flame Spead
330800
Number of floors with minimum damage
401128
N

In [57]:
df_fire['Number of floors with significant damage']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
407709   NaN
407710   NaN
407711   NaN
407712   NaN
407713   NaN
Name: Number of floors with significant damage, dtype: float64

In [44]:
df_fire['Estimated Property Loss'].value_counts()

 0.0         111543
 50.0          2637
 1000.0        1133
 500.0         1056
 5000.0         934
              ...  
 535000.0         1
 10700.0          1
 10570.0          1
-10.0             1
 9841.0           1
Name: Estimated Property Loss, dtype: int64